<a href="https://colab.research.google.com/github/MatheusMGirardi/py-spark-olympic-athlete-data/blob/main/PySpark_big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

heesoo37_120_years_of_olympic_history_athletes_and_results_path = kagglehub.dataset_download('heesoo37/120-years-of-olympic-history-athletes-and-results')
blauzo_olympic_data_no_commas_for_rdd_path = kagglehub.dataset_download('blauzo/olympic-data-no-commas-for-rdd')

print('Data source import complete.')


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil

In [ ]:
!pip install -q findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

if __name__ == "__main__":
    spark = SparkSession.builder.appName("TDE4").master("local").getOrCreate()

#### 1. CSV loading

In [ ]:
df = spark.read.csv("/kaggle/input/olympic-data-no-commas-for-rdd/athletes.csv")
rdd = df.rdd

In [ ]:
for row in rdd.take(2):
    print(row)

Row(_c0='ID', _c1='Name', _c2='Sex', _c3='Age', _c4='Height', _c5='Weight', _c6='Team', _c7='NOC', _c8='Games', _c9='Year', _c10='Season', _c11='City', _c12='Sport', _c13='Event', _c14='Medal')
Row(_c0='1', _c1='A Dijiang', _c2='M', _c3='24', _c4='180', _c5='80', _c6='China', _c7='CHN', _c8='1992 Summer', _c9='1992', _c10='Summer', _c11='Barcelona', _c12='Basketball', _c13="Basketball Men's Basketball", _c14='NA')


In [ ]:
# remove o header
rdd = rdd.zipWithIndex().filter(lambda row_index: row_index[1] > 0).map(lambda row_index: row_index[0])

#### 1.1 Contagem de Atletas por Ano

In [ ]:
consulta1_1m = rdd.map(lambda row: ((row._c9), 1))
consulta1_1r = consulta1_1m.reduceByKey(lambda a,b: a + b)

In [ ]:
results = consulta1_1r.take(10)
for result in results:
    print(result)

('1992', 16413)
('2012', 12920)
('1920', 4292)
('1900', 1936)
('1988', 14676)
('1994', 3160)
('1932', 3321)
('2002', 4109)
('1952', 9358)
('1980', 8937)


In [ ]:
output_dir = "/kaggle/working/consulta1_2"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

In [ ]:
consulta1_1r.saveAsTextFile(output_dir)

### 1.2 Contagem de Atletas por esporte

In [ ]:
consulta1_2m = rdd.map(lambda row: ((row._c12), 1))
consulta1_2r = consulta1_2m.reduceByKey(lambda a,b: a + b) #soma os val

In [ ]:
results = consulta1_2r.take(10)
for result in results:
    print(result)

('Basketball', 4536)
('Judo', 3801)
('Football', 6745)
('Tug-Of-War', 170)
('Speed Skating', 5613)
('Cross Country Skiing', 9133)
('Athletics', 38624)
('Ice Hockey', 5516)
('Swimming', 23195)
('Badminton', 1457)


In [ ]:
output_dir = "/kaggle/working/consulta1_2"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

In [ ]:
consulta1_r.saveAsTextFile("/kaggle/working/consulta1_2")

### 1.3 Média de Idade dos Atletas por Gênero

In [ ]:
consulta1_3m = rdd.map(lambda row: ((row._c12), 1))
consulta1_3r = consulta1_2m.reduceByKey(lambda a,b: a + b) #soma os val